In [527]:
import pysam
import numpy as np
from Bio import SeqIO
import time 
import matplotlib.patches as patches
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import itertools
import pandas as pd 
from tabulate import tabulate
import csv
import random

np.set_printoptions(threshold=np.inf)
min_quality_score = 8

#Load the bam file 
bamfile = pysam.AlignmentFile(
    "/Data2/01_09_24_R1041_DiMeLo_MsIgg/01_09_24_R1041_DiMeLo_MsIgg_5mC_6mA_winnowmap_sorted_sorted_clean_filtered.bam",
    "rb") 


assembly_ = open("/Data1/hg002v1.0.1.fasta", "r")

start_time = time.time()

#Load the reference genome and make it into a dictionary 
fasta_sequences = SeqIO.parse(assembly_, "fasta")
assembly={}
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    assembly[name] = sequence

#Make a dictionary for all the chromosomes and their corresponding sequence length 
assembly_sequence_length = {}    
for chromosome in assembly:
    assembly_sequence_length[chromosome] = len(assembly[chromosome])
    
end_time = time.time()
elapsed_time = end_time - start_time
print (elapsed_time, "seconds")
assembly_.close()



40.48095941543579 seconds


In [168]:
'''The purpose of the code here is to make an active array where the chromosome name is
the dictionary key. The active array blocks are in lists inside the key.
'''
import os
input_file = '/Data1/HG002v0.1censat/hg002v1.0.fasta.manualAlpha.cenSat_H1L_merged.bed'
active_dict = {}
with open(input_file, 'r') as infile:  
    num = 0 
    previous_chr_num = ""
    previous_parental_status = ""
    for i in infile:
        chr_num = i.split ('_')[0]
        parental_status = i.split ('_')[1][0:8]
        if chr_num == previous_chr_num and parental_status !=previous_parental_status:
            num = 0 
        elif chr_num == previous_chr_num and parental_status ==previous_parental_status:
            pass 
        else: 
            num = 0 
        active = i.split ()
        if active[0] not in active_dict:
            
            active_dict[active[0]] = [[int(active[1]) ,int(active[2])]]
        else:
            active_dict[active[0]].append([int(active[1]) ,int(active[2])])
        previous_chr_num = chr_num 
        previous_parental_status = parental_status
#print (active_dict)


In [326]:
''' here in the code, I am formulating the CDR regions and listing the CDRs in each and every chromosome'''
input_file = '/Data1/CDRStrictBedFiles/hg002v1.merged.strict.CDR.bed'
CDR_dict = {}
with open(input_file, 'r') as infile:  
    for i in infile:
        chr_num = i.split('\t')[0]
        CDR_start = i.split('\t')[1]
        CDR_end = i.split('\t')[2].split('\n')[0]
        if chr_num not in CDR_dict:
            CDR_dict[chr_num] = [[CDR_start ,CDR_end]]
        elif chr_num in CDR_dict:  
            CDR_dict[chr_num].append ([CDR_start ,CDR_end])

#print (CDR_dict)
'''This piece of code is to determine how many CDRs there are in a given chromosome'''
for i in CDR_dict:
#    print (i, len(CDR_dict[i]))
    for CDR in CDR_dict[i]:
        CDR_length = int(CDR[1]) - int(CDR[0])
        total_A_count = assembly[i][int(CDR[0]):int(CDR[1])].count ("A")


In [327]:
'''Based on the CDR regions, I am obtaining CDR adjacent regions in the same format as the CDR data set above'''
CDR_adjacent = {}
for chromosome in CDR_dict: 
    CDR_adjacent[chromosome] =[]
    for CDR in CDR_dict[chromosome]: 
        #print (CDR) 
        CDR_adjacent_left_space = [int(CDR[0]) - 1001, int(CDR[0]) - 1]
        CDR_adjacent_right_space = [int(CDR[1]) + 1, int(CDR[1]) + 1001]
        
        CDR_adjacent[chromosome].append (CDR_adjacent_left_space)
        CDR_adjacent[chromosome].append (CDR_adjacent_right_space)
        
    
        
#print (CDR_adjacent)

In [331]:
chrom_X_CDR_dict ={}
chrom_X_CDR_adjacent_dict = {}
chrom_X_CDR_dict['chrX_MATERNAL'] = CDR_dict['chrX_MATERNAL']
chrom_X_CDR_adjacent_dict ['chrX_MATERNAL'] = CDR_adjacent['chrX_MATERNAL']
print (chrom_X_CDR_adjacent_dict)

{'chrX_MATERNAL': [[59281767, 59282767], [59299768, 59300768], [59414767, 59415767], [59448768, 59449768], [59460767, 59461767], [59496768, 59497768]]}


In [349]:
def chromosome_arm_random_region (segment_num, chr_name, H1L_active_dict, assembly_sequence_length): 
    # create variables to contain regions chosen 
    chromosome_arm_regions = []
    excluded_portion = []
    H1L_active_length = 0 
    
    for active_region in H1L_active_dict[chr_name]: 
        #calculate the length of the active array region for each chromosome 
        H1L_active_length = active_region[1] - active_region[0]
        
        #defining where the chromosome arm regions are
        
        #The left side of the active array 
        chromosome_arm_regions.append([0,active_region[0] - 1])
        
        #The right side of the active array 
        chromosome_arm_regions.append([active_region[1] + 1,assembly_sequence_length[chr_name]])
        
        #calculate the percentage portion of where the active array is in and add them to the exclusion bin 
        start_portion = active_region[0] / H1L_active_length
        end_portion = active_region[1] / H1L_active_length
        excluded_portion.append (int(start_portion))
        excluded_portion.append (int(end_portion))
        
    #calculate the total portions of the active array 
    total_segment_amount = int(assembly_sequence_length[chr_name] / H1L_active_length)
    
    
    #pick defined amount of random numbers between 0 and the pre defined amount of random numbers 
    random_numbers = []
    for num in range(segment_num):
        while True: 
            #if the same random number gets picked twice, repeat 
            current_random_number = random.randint(0, total_segment_amount)
            if current_random_number not in (excluded_portion and random_numbers):
                break
            
            
        random_numbers.append(current_random_number)
    
    #Expand the chromosome portion number to chromosome position number by multiplying 
    random_picked_regions = [num * H1L_active_length for num in random_numbers]
    uncoded_region_list = []
    
    #Make a dictionary that contains randomly picked region for each chromosome 
    for item in random_picked_regions: 
        arms_region_start = item
        arms_region_end = item + H1L_active_length
        uncoded_region_list.append([arms_region_start, arms_region_end])
        
    return uncoded_region_list


chromosome_arm_random_region_dict = {}
for chromosome in chrom_X_CDR_dict: 
    chromosome_arm_random_region_dict[chromosome] = chromosome_arm_random_region (5,
                                                                             chromosome, 
                                                                             active_dict, 
                                                                             assembly_sequence_length)


In [524]:
#test_dict = {'chrX_MATERNAL':[['52000000','62000000']]}
#scenario_1 = {'chrX_MATERNAL':[['51900000','51900810']]}
#scenario_2 = {'chrX_MATERNAL':[['51900810','51901200']]}
#scenario_3 = {'chrX_MATERNAL':[['52167000','52167900']]}
#scenarios = [scenario_1,scenario_2,scenario_3]
chromosome_arm_random_region_dict_chrX = {'chrX_MATERNAL': [[37350684, 40463241], [6225114, 9337671], [143177622, 146290179], [68476254, 71588811]]}
print (chromosome_arm_random_region_dict_chrX)
print (chrom_X_CDR_dict)
print (chrom_X_CDR_adjacent_dict)
chromX = [chrom_X_CDR_dict,chromosome_arm_random_region_dict_chrX,chrom_X_CDR_adjacent_dict]

{'chrX_MATERNAL': [[37350684, 40463241], [6225114, 9337671], [143177622, 146290179], [68476254, 71588811]]}
{'chrX_MATERNAL': [['59282768', '59299767'], ['59415768', '59448767'], ['59461768', '59496767']]}
{'chrX_MATERNAL': [[59281767, 59282767], [59299768, 59300768], [59414767, 59415767], [59448768, 59449768], [59460767, 59461767], [59496768, 59497768]]}


In [525]:
''' 
The input of the function is a dictionary in the format of 'chromosome':[[start,end],[start,end]] 
'''
def region_read_mA_density_calculator (chromosome_coordinates,name): 
    data_table = [] 

    #get each chromosome
    for chr_name in chromosome_coordinates:

        for region in chromosome_coordinates[chr_name]:
            region_density = []

            region_start_index = int(region[0])
            region_end_index = int(region[1])
            
            for read in bamfile.fetch(chr_name,region_start_index,region_end_index):
                #make an if statement to check a specific read front, middle, end regions 
                #setting read start, end, density, length variables 
                    
                #Get the starting and ending positions of the reads 
                read_start_position = read.reference_start
                read_end_position = read.reference_end
                read_density = 0 
        
                #Get sequence information which shows deletions and insertions 
                sequence = read.get_aligned_pairs(matches_only=False, with_seq = True)

                #make a numpy of the sequence length which eliminates the deletion
                sequence_insertions = []
                sequence_deletions = [] 
                
                true_sequence = ''
                
                for item in sequence:
                    if item[0] is None:
                        sequence_deletions.append (item[1] - read_start_position)
                    elif item[1] is None:
                        sequence_insertions.append (item[0])
                    else: 
                        true_sequence+=item[2]
                
                #take sequence length excluding insertions 
                insertion_amount = sum(1 for item in sequence if item[1] is None)
                sequence_length = len(sequence) - insertion_amount
                
                #removing reads shorter than 50000 
                if sequence_length < 50000:
                    continue 

                #make a mod np array with the length of the read length
                mod=read.modified_bases_forward
                
                #make a mod score with its original length 
                mod_score = np.zeros(len(read.query_sequence),)
                
                #make transfer mA positions to mod np array corresponded to their sequence positions 
                try:
                    for indices, values in mod[('A', 0, 'a')]:
                        mod_score[indices] = values

                # No mod would return KeyError 
                except KeyError:
                    continue
                    
                
                #remove insertions 

                mod_score_insertions_removed = np.delete(mod_score, sequence_insertions)

                #insert deletions 
                if len(sequence_deletions) == 0: 
                    mod_score_deletions_inserted = mod_score_insertions_removed

                elif len(mod_score_insertions_removed) < max(sequence_deletions):

                    mod_score_deletions_inserted_padded = np.pad(
                        mod_score_insertions_removed, (0, 
                                                        sequence_length - len(mod_score_insertions_removed)),
                                                    constant_values=0)


                    mod_score_deletions_inserted = np.insert(mod_score_deletions_inserted_padded,
                                                             sequence_deletions,
                                                             0)
                else: 
                    mod_score_deletions_inserted = np.insert(mod_score_insertions_removed,
                                                             sequence_deletions,
                                                             0)
                mod_score = mod_score_deletions_inserted
                
                if (region_end_index > read_end_position) and (region_start_index > read_start_position): 
                    mod_start = 0
                    mod_end = sequence_length
                
                # scenario 1: when the defined region is inside the read
                elif (read_start_position <= region_start_index) and (read_end_position >= region_end_index):
                    mod_start = region_start_index - read_start_position
                    mod_end = region_end_index - read_start_position
                
                # scenario 3: when the defined region covers a bit of the end of the read
                elif (read_end_position < region_end_index) and (read_end_position > region_start_index):
                    mod_start = region_start_index - read_start_position 
                    mod_end = sequence_length
                
                # scenario 2: when the defined region covers a bit of the beginning of the read
                elif (read_start_position > region_start_index) and (read_start_position < region_end_index):
                    mod_start = 0
                    mod_end = region_end_index - read_start_position 
                
                #use the defined starting and ending positons in the region to subset mod numpy
                trimmed_mod_score = mod_score[mod_start:mod_end]
                
                #removing all the zeros 
                mod_no_zeros = trimmed_mod_score[trimmed_mod_score != 0]
                mA = len (mod_no_zeros)
                
                #Getting the total amount of As in the subsetted region of the sequence 
                total_A = true_sequence[mod_start:mod_end].count("A")
                
                
                #calculate read density
                try:
                    read_density = mA / total_A
                    
                except ZeroDivisionError:
                    pass
                region_density.append (read_density)

                
                    
                
            #calculate averaged region density average 
            try:
                region_density_average = sum(region_density)/len(region_density)
            
            except ZeroDivisionError:
                region_density_average = 0
            data_table.append ([chr_name,region,region_density_average])

        
    table = tabulate(data_table, headers="firstrow", tablefmt="fancy_grid", floatfmt=".18f")
    
    filename = f"{name}_region_density_scores.csv"
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile,delimiter="\t")
        writer.writerows(data_table)

    print (name)
    print (table)

#region_read_mA_density_calculator(test_dict,'test_dict')

#for scenario in scenarios: 
    #region_read_mA_density_calculator (scenario,str(scenario))
    

In [529]:
from joblib import Parallel, delayed
all_dicts = [chromosome_arm_random_region_dict, CDR_adjacent, CDR_dict]
all_names = ['chromosome_arm_random_region_dict', 'CDR_adjacent', 'CDR_dict']
def get_variable_name(var, locals_dict):
    for name, value in locals_dict.items():
        if value is var:
            return name
    return None



#results=Parallel(n_jobs=3)(delayed(region_read_mA_density_calculator)(all_dicts,all_names) for i in range(3))

for dictionary in range (0, len (chromX)): 
    variable_name = get_variable_name(chromX[dictionary], locals())

    region_read_mA_density_calculator (chromX[dictionary],str(f"{variable_name}_Ms"))

chrom_X_CDR_dict_Ms
╒═════════════════╤════════════════════════════╤═════════════════════════╕
│ chrX_MATERNAL   │ ['59282768', '59299767']   │   0.0005460337404030495 │
╞═════════════════╪════════════════════════════╪═════════════════════════╡
│ chrX_MATERNAL   │ ['59415768', '59448767']   │    0.001174757070153008 │
├─────────────────┼────────────────────────────┼─────────────────────────┤
│ chrX_MATERNAL   │ ['59461768', '59496767']   │    0.000886125639382954 │
╘═════════════════╧════════════════════════════╧═════════════════════════╛
chromosome_arm_random_region_dict_chrX_Ms
╒═════════════════╤════════════════════════╤═════════════════════════╕
│ chrX_MATERNAL   │ [37350684, 40463241]   │   0.0015864973155825344 │
╞═════════════════╪════════════════════════╪═════════════════════════╡
│ chrX_MATERNAL   │ [6225114, 9337671]     │    0.001385727181242197 │
├─────────────────┼────────────────────────┼─────────────────────────┤
│ chrX_MATERNAL   │ [143177622, 146290179] │    0.00122879

'''version 1 '''
''' 
The input of the function is a dictionary in the format of 'chromosome':[[start,end],[start,end]] 
'''
def region_read_mA_density_calculator (chromosome_coordinates,name): 
    data_table = [] 

    #get each chromosome
    for chr_name in chromosome_coordinates:

        for region in chromosome_coordinates[chr_name]:
            region_density = []

            region_start_index = int(region[0])
            region_end_index = int(region[1])
            for read in bamfile.fetch(chr_name,region_start_index,region_end_index):
                #checked read amoounts and names √
                #make an if statement to check a specific read front, middle, end regions 
                if read.query_name == 'c06221ac-7a04-496c-87aa-e8f6bce21c34':

                #setting read start, end, density, length variables 
                    
                    #Get the starting and ending positions of the reads 
                    read_start_position = read.reference_start
                    read_end_position = read.reference_end
                    
                    print ('start:', read_start_position)
                    print ('end:', read_end_position)
                    
                    read_density = 0 
                    
                    
                    sequence = read.get_aligned_pairs(matches_only=False, with_seq = True)
                    count_none_first = sum(1 for item in sequence if item[1] is None)
                    
                    
                    print ('sequence item:',count_none_first)
                    sequence_length = len(sequence) - count_none_first
                    print ('recalculated sequence length:', sequence_length)

                    #removing reads shorter than 50000 
                    if sequence_length < 50000:
                        continue 

                    #make a mod np array with the length of the read length
                    mod=read.modified_bases_forward
                    mod_score = np.zeros(sequence_length,)


                    #make transfer mA positions to mod np array corresponded to their sequence positions 
                    try:
                        for indices, values in mod[('A', 0, 'a')]:
                            mod_score[indices] = values

                    except KeyError:
                        continue 

                    #last thing to check is the region cutter 
                    if (read_start_position <= region_start_index) and (read_end_position >= region_end_index):
                        print ("scenario 1: when the defined region is inside the read")
                        mod_start = region_start_index - read_start_position
                        mod_end = region_end_index - read_start_position
                        
                    elif (read_end_position < region_end_index) and (read_end_position > region_start_index):
                        print ("scenario 3: when the defined region covers a bit of the end of the read")
                        mod_start = region_start_index - read_start_position 
                        mod_end = sequence_length
                    elif (read_start_position > region_start_index) and (read_start_position < region_end_index):
                        print ("scenario 2: when the defined region covers a bit of the beginning of the read")
                        mod_start = 0
                        mod_end = region_end_index - read_start_position 
                    else: 
                        mod_start = 0
                        mod_end = sequence_length
                    
                    
                    print (mod_start,mod_end)
                    trimmed_mod_score = mod_score[mod_start:mod_end]
                    trimmed_sequence = read.query_sequence[mod_start:mod_end]
                    print (trimmed_sequence)
                    mod_threshold = 200
                    mask = trimmed_mod_score <= mod_threshold
                    trimmed_mod_score[mask] = 0 
                    mod_no_zeros = trimmed_mod_score[trimmed_mod_score != 0]
                    mA = len(mod_no_zeros)
                    #checked filtered mA amount when reads are not sectioned into regions √

                    # should we do total A in the assembly or use the total A amount ? 

                    #total_A = assembly[chr_name][(mod_start+read_start_position):(mod_end+read_start_position)].count("A")
                    total_A = read.query_sequence[mod_start:mod_end].count("A")
                    try:
                        read_density = mA / total_A
                    except ZeroDivisionError:
                        read_density = 0
                    region_density.append (read_density)

                    print (region_density)
                    

                

            try:
                region_density_average = sum(region_density)/len(region_density)
            except ZeroDivisionError:
                region_density_average = 0
            data_table.append ([chr_name,region,region_density_average])
            
            
    table = tabulate(data_table, headers="firstrow", tablefmt="fancy_grid", floatfmt=".18f")

    filename = f"{name}_region_density_scores.csv"
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile,delimiter="\t")
        writer.writerows(data_table)

    print (name)
    print (table)

#region_read_mA_density_calculator(test_dict,'test_dict')

for scenario in scenarios: 
    region_read_mA_density_calculator (scenario,str(scenario))
    

In [180]:
def get_variable_name(var, locals_dict):
    for name, value in locals_dict.items():
        if value is var:
            return name
    return None

# Example usage
my_variable = 42
variable_name = get_variable_name(my_variable, locals())
print(f"The name of the variable is: {variable_name}")


The name of the variable is: my_variable


In [ ]:
''' 
version 2 
'''
def region_read_mA_density_calculator (chromosome_coordinates,name): 
    data_table = [] 

    #get each chromosome
    for chr_name in chromosome_coordinates:

        for region in chromosome_coordinates[chr_name]:
            region_density = []

            region_start_index = int(region[0])
            region_end_index = int(region[1])
            for read in bamfile.fetch(chr_name,region_start_index,region_end_index):
                #checked read amoounts and names √
                #make an if statement to check a specific read front, middle, end regions 

                #setting read start, end, density, length variables 
                    
                #Get the starting and ending positions of the reads 
                read_start_position = read.reference_start
                read_end_position = read.reference_end

                read_density = 0 
        
                #Get sequence information which shows deletions and insertions 
                sequence = read.get_aligned_pairs(matches_only=False, with_seq = True)
                
                #take sequence length excluding insertions 
                insertion_amount = sum(1 for item in sequence if item[1] is None)
                sequence_length = len(sequence) - insertion_amount
                #sequence_length = len(sequence)

                #removing reads shorter than 50000 
                if sequence_length < 50000:
                    continue 

                #make a mod np array with the length of the read length
                mod=read.modified_bases_forward
                mod_score = np.zeros(sequence_length,)
                


                #make transfer mA positions to mod np array corresponded to their sequence positions 
                try:
                    for indices, values in mod[('A', 0, 'a')]:
                        mod_score[indices] = values
                
                # No mod would return KeyError 
                except KeyError:
                    continue 
                    
                except IndexError:
                    print (mod[('A', 0, 'a')])
                    
                    print ("mA mod length:",mod[('A', 0, 'a')][-1])
                    print ("mod_Score length:",len(mod_score))
                    print ('read_start_position', read_start_position)
                    print ('read_end_position', read_end_position)
                    print ('difference', read_end_position - read_start_position)
                    

                # scenario 1: when the defined region is inside the read
                if (read_start_position <= region_start_index) and (read_end_position >= region_end_index):
                    mod_start = region_start_index - read_start_position
                    mod_end = region_end_index - read_start_position
                
                # scenario 3: when the defined region covers a bit of the end of the read
                elif (read_end_position < region_end_index) and (read_end_position > region_start_index):
                    mod_start = region_start_index - read_start_position 
                    mod_end = sequence_length
                
                # scenario 2: when the defined region covers a bit of the beginning of the read
                elif (read_start_position > region_start_index) and (read_start_position < region_end_index):
                    mod_start = 0
                    mod_end = region_end_index - read_start_position 
                
                # scenario 4: defined region covers the entire read 
                else: 
                    mod_start = 0
                    mod_end = sequence_length

                #use the defined starting and ending positons in the region to subset mod numpy
                trimmed_mod_score = mod_score[mod_start:mod_end]
                
                #removing all the zeros 
                mod_no_zeros = trimmed_mod_score[trimmed_mod_score != 0]
                
                #Getting the total amount of As in the subsetted region of the sequence 
                total_A = read.query_sequence[mod_start:mod_end].count("A")
                
                #calculate read density
                try:
                    read_density = mA / total_A
                except ZeroDivisionError:
                #if there is no A in the region, density becomes 0 
                    read_density = 0
                region_density.append (read_density)
                

            #calculate averaged region density average 
            try:
                region_density_average = sum(region_density)/len(region_density)
            
            except ZeroDivisionError:
                region_density_average = 0
            data_table.append ([chr_name,region,region_density_average])
            
            
    table = tabulate(data_table, headers="firstrow", tablefmt="fancy_grid", floatfmt=".18f")

    filename = f"{name}_region_density_scores.csv"
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile,delimiter="\t")
        writer.writerows(data_table)

    print (name)
    print (table)

#region_read_mA_density_calculator(test_dict,'test_dict')

#for scenario in scenarios: 
    #region_read_mA_density_calculator (scenario,str(scenario))
    

In [277]:
import numpy as np

# Create a NumPy array with None values
my_array = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=object)

# List of indices to add items
indices_to_add = [1, 5, 8, 12]

# Values to add at corresponding indices
values_to_add = [11, 12, 13, 14]

# Determine the desired length for the array
desired_length = max(indices_to_add) + 1

# Pad the array with np.nan values to the desired length
padded_array = np.pad(my_array, (0, desired_length - len(my_array)), constant_values=np.nan)

# Use NumPy's insert function to add items at specified indices
result_array = np.insert(padded_array, indices_to_add, values_to_add)

print(result_array)


[0 11 1 2 3 4 12 5 6 7 13 8 9 nan nan 14 nan]


In [ ]:
''' 
version 3 
'''
def region_read_mA_density_calculator (chromosome_coordinates,name): 
    data_table = [] 

    #get each chromosome
    for chr_name in chromosome_coordinates:

        for region in chromosome_coordinates[chr_name]:
            region_density = []

            region_start_index = int(region[0])
            region_end_index = int(region[1])
            for read in bamfile.fetch(chr_name,region_start_index,region_end_index):
                #checked read amoounts and names √
                #make an if statement to check a specific read front, middle, end regions 

                #setting read start, end, density, length variables 
                    
                #Get the starting and ending positions of the reads 
                read_start_position = read.reference_start
                read_end_position = read.reference_end

                read_density = 0 
        
                #Get sequence information which shows deletions and insertions 
                sequence = read.get_aligned_pairs(matches_only=False, with_seq = True)
                #make a numpy of the sequence length which eliminates the deletion
                
                print ('sequence length_get_aligned_pairs:', len(read.get_aligned_pairs(matches_only=False, with_seq = True)) - sum(1 for item in sequence if item[0] is None))
                print ('sequence forward sequence:', len(read.query_sequence))
                       
                #take sequence length excluding insertions 
                insertion_amount = sum(1 for item in sequence if item[1] is None)
                sequence_length = len(sequence) - insertion_amount
                
                
                

                #removing reads shorter than 50000 
                if sequence_length < 50000:
                    continue 

                #make a mod np array with the length of the read length
                mod=read.modified_bases_forward
                mod_score = np.zeros(sequence_length,)
                
                #make a mod score with its original length 

                #make transfer mA positions to mod np array corresponded to their sequence positions 
                try:
                    for indices, values in mod[('A', 0, 'a')]:
                        mod_score[indices] = values
                
                #eliminate the insertions in the mod score numpy 
                
                # No mod would return KeyError 
                except KeyError:
                    continue 
                    
                except IndexError:
                    print (mod[('A', 0, 'a')])
                    sequence_length = len(read.get_aligned_pairs(matches_only=False, with_seq = True))
                    print ('sequence length:', len(read.get_aligned_pairs(matches_only=False, with_seq = True)))
                    print ("mA mod length:",mod[('A', 0, 'a')][-1])
                    print ("mod_Score length:",len(mod_score))
                    print ('read_start_position', read_start_position)
                    print ('read_end_position', read_end_position)
                    print ('difference', read_end_position - read_start_position)
                    

                # scenario 1: when the defined region is inside the read
                if (read_start_position <= region_start_index) and (read_end_position >= region_end_index):
                    mod_start = region_start_index - read_start_position
                    mod_end = region_end_index - read_start_position
                
                # scenario 3: when the defined region covers a bit of the end of the read
                elif (read_end_position < region_end_index) and (read_end_position > region_start_index):
                    mod_start = region_start_index - read_start_position 
                    mod_end = sequence_length
                
                # scenario 2: when the defined region covers a bit of the beginning of the read
                elif (read_start_position > region_start_index) and (read_start_position < region_end_index):
                    mod_start = 0
                    mod_end = region_end_index - read_start_position 
                
                # scenario 4: defined region covers the entire read 
                else: 
                    mod_start = 0
                    mod_end = sequence_length

                #use the defined starting and ending positons in the region to subset mod numpy
                trimmed_mod_score = mod_score[mod_start:mod_end]
                
                #removing all the zeros 
                mod_no_zeros = trimmed_mod_score[trimmed_mod_score != 0]
                
                #Getting the total amount of As in the subsetted region of the sequence 
                total_A = read.query_sequence[mod_start:mod_end].count("A")
                
                #calculate read density
                try:
                    read_density = mA / total_A
                except ZeroDivisionError:
                #if there is no A in the region, density becomes 0 
                    read_density = 0
                region_density.append (read_density)
                

            #calculate averaged region density average 
            try:
                region_density_average = sum(region_density)/len(region_density)
            
            except ZeroDivisionError:
                region_density_average = 0
            data_table.append ([chr_name,region,region_density_average])
            
            
    table = tabulate(data_table, headers="firstrow", tablefmt="fancy_grid", floatfmt=".18f")

    filename = f"{name}_region_density_scores.csv"
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile,delimiter="\t")
        writer.writerows(data_table)

    print (name)
    print (table)

#region_read_mA_density_calculator(test_dict,'test_dict')

#for scenario in scenarios: 
    #region_read_mA_density_calculator (scenario,str(scenario))
    